# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [98]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import numpy as np

from patsy import dmatrices
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

In [76]:
df = pd.read_csv('previsao_de_renda.csv')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [78]:
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [79]:
print(df.columns)

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')


In [80]:
#Apagando colunas
df = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])

In [81]:
# Transforme variáveis categóricas em dummies
df_dummies = pd.get_dummies(df, drop_first=False)

# Verifique se todas as colunas agora são numéricas
print(df_dummies.dtypes)

posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
sexo_F                              bool
sexo_M                              bool
tipo_renda_Assalariado              bool
tipo_renda_Bolsista                 bool
tipo_renda_Empresário               bool
tipo_renda_Pensionista              bool
tipo_renda_Servidor público         bool
educacao_Primário                   bool
educacao_Pós graduação              bool
educacao_Secundário                 bool
educacao_Superior completo          bool
educacao_Superior incompleto        bool
estado_civil_Casado                 bool
estado_civil_Separado               bool
estado_civil_Solteiro               bool
estado_civil_União                  bool
estado_civil_Viúvo                  bool
tipo_residencia_

In [82]:
df_dummies = df_dummies.dropna()

In [83]:
df_dummies.dtypes

posse_de_veiculo                    bool
posse_de_imovel                     bool
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
sexo_F                              bool
sexo_M                              bool
tipo_renda_Assalariado              bool
tipo_renda_Bolsista                 bool
tipo_renda_Empresário               bool
tipo_renda_Pensionista              bool
tipo_renda_Servidor público         bool
educacao_Primário                   bool
educacao_Pós graduação              bool
educacao_Secundário                 bool
educacao_Superior completo          bool
educacao_Superior incompleto        bool
estado_civil_Casado                 bool
estado_civil_Separado               bool
estado_civil_Solteiro               bool
estado_civil_União                  bool
estado_civil_Viúvo                  bool
tipo_residencia_

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [84]:
# Ajuste a coluna de acordo com os nomes presentes no seu dataset
X = df_dummies.drop(columns="renda")
y = df_dummies["renda"]

In [85]:
#Separação dos dados em treinamento (75%) e teste (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [86]:
# Com regularização ridge com alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
best_r2 = float('-inf')
best_alpha = None

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f'Alpha: {alpha}, R^2: {r2}')

    if r2 > best_r2:
        best_r2 = r2
        best_alpha = alpha

print(f"O melhor modelo tem alpha = {best_alpha} com R^2 = {best_r2}")

Alpha: 0, R^2: 0.29788795819542546
Alpha: 0.001, R^2: 0.2979664398824702
Alpha: 0.005, R^2: 0.2979665922694845
Alpha: 0.01, R^2: 0.2979667826039293
Alpha: 0.05, R^2: 0.2979682993456856
Alpha: 0.1, R^2: 0.29797018062542413
O melhor modelo tem alpha = 0.1 com R^2 = 0.29797018062542413


In [87]:
#Com regressão LASSO
best_r2_lasso = float("-inf")
best_alpha_lasso = None

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    print(f"Alpha: {alpha}, R^2: {r2}")

    if r2 > best_r2_lasso:
        best_r2_lasso = r2
        best_alpha_lasso = alpha

print(f"O melhor modelo LASSO tem alpha = {best_alpha_lasso} com R^2 = {best_r2_lasso}")

C:\Users\Joel\AppData\Local\Temp\ipykernel_5156\522920222.py:7: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  lasso.fit(X_train, y_train)
C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.036e+11, tolerance: 8.060e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha: 0, R^2: 0.2979667288792085
Alpha: 0.001, R^2: 0.297967209660144


C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.859e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.280e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.005, R^2: 0.29796908589432314
Alpha: 0.01, R^2: 0.2979714395605716


C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.770e+11, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.818e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.05, R^2: 0.2979901829552367
Alpha: 0.1, R^2: 0.29800172533121116
O melhor modelo LASSO tem alpha = 0.1 com R^2 = 0.29800172533121116


C:\Users\Joel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.642e+10, tolerance: 8.060e+07
  model = cd_fast.enet_coordinate_descent(


In [88]:
X = X.apply(pd.to_numeric, errors='coerce')
X = X.astype(int)

In [89]:
def stepwise_selection(X, y, 
                       initial_list=None, 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    """ 
    Realiza uma seleção de variáveis para frente e para trás
    baseada no p-valor do `statsmodels.api.OLS`.
    
    Argumentos:
        X - pandas.DataFrame com as variáveis candidatas
        y - lista ou array com o alvo
        initial_list - lista de variáveis para começar (nomes das colunas de X)
        threshold_in - incluir uma variável se seu p-valor < threshold_in
        threshold_out - excluir uma variável se seu p-valor > threshold_out
        verbose - se deve imprimir a sequência de inclusões e exclusões
    
    Retorna: lista de variáveis selecionadas
    Sempre defina threshold_in < threshold_out para evitar loop infinito.
    Veja https://en.wikipedia.org/wiki/Stepwise_regression para mais detalhes
    """
    if initial_list is None:
        initial_list = []
    
    included = list(initial_list)
    
    while True:
        changed = False
        
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.float64)
        for new_column in excluded:
            if new_column in X.columns:  # Verificação adicional
                model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
                new_pval[new_column] = model.pvalues[new_column]
            else:
                new_pval[new_column] = np.nan  # Atribua NaN se a coluna não existir
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6f}'.format(best_feature, best_pval))
        
        # Backward step
        if len(included) > 0:
            model = sm.OLS(y, sm.add_constant(X[included])).fit()
            pvalues = model.pvalues.iloc[1:]  # Use all coefs except intercept
            if not pvalues.empty:
                worst_pval = pvalues.max()  # Null if pvalues is empty
                if worst_pval > threshold_out:
                    worst_feature = pvalues.idxmax()
                    included.remove(worst_feature)
                    changed = True
                    if verbose:
                        print('Drop {:30} with p-value {:.6f}'.format(worst_feature, worst_pval))
        
        if not changed:
            break
    
    return included

In [90]:
# Supondo que X e y estejam definidos e preparados corretamente
resultado = stepwise_selection(X, y)

print('Variáveis resultantes:')
print(resultado)

Add  tempo_emprego                  with p-value 0.000000
Add  sexo_F                         with p-value 0.000000
Add  sexo_M                         with p-value 0.000000
Add  tipo_renda_Empresário          with p-value 0.000000
Add  idade                          with p-value 0.000000
Add  educacao_Superior completo     with p-value 0.000004
Add  qt_pessoas_residencia          with p-value 0.006468
Variáveis resultantes:
['tempo_emprego', 'sexo_F', 'sexo_M', 'tipo_renda_Empresário', 'idade', 'educacao_Superior completo', 'qt_pessoas_residencia']


In [91]:
# Ajustando o Modelo 1
model1 = smf.ols('renda ~ C(sexo) + C(posse_de_imovel) + np.log(tempo_emprego)', data=df).fit()

# Ajustando o Modelo 2
model2 = smf.ols("np.log(renda) ~ C(sexo, Treatment(reference='M')) + C(posse_de_veiculo, Treatment(reference=True)) + C(posse_de_imovel, Treatment(reference=True))  + idade + tempo_emprego + qt_pessoas_residencia", data=df).fit()

In [92]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     913.7
Date:                Mon, 26 Aug 2024   Prob (F-statistic):               0.00
Time:                        14:33:42   Log-Likelihood:            -1.2939e+05
No. Observations:               12427   AIC:                         2.588e+05
Df Residuals:                   12423   BIC:                         2.588e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                  -1031.8820    183.627     -5.619      0.000   -1391.820    -671.945
C(sexo)[T.M]                5551.7426    150.939     36.781      0.000    5255.878    5847.607
C(posse_de_imovel)[T.True]   444.9310    151.797      2.931      0.003     147.384     742.478
np.log(tempo_emprego)       2954.4481     72.447     40.781      0.000    2812.441    3096.455
==============================================================================
Omnibus:                    17867.928   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10069030.831
Skew:                           8.415   Prob(JB):                         0.00
Kurtosis:                     141.430   Cond. No.                         6.84
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1097.
Date:                Mon, 26 Aug 2024   Prob (F-statistic):               0.00
Time:                        14:33:45   Log-Likelihood:                -13674.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===========================================================================================================================
                                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
Intercept                                                   8.1150      0.040    205.375      0.000       8.038       8.192
C(sexo, Treatment(reference='M'))[T.F]                     -0.7681      0.015    -52.652      0.000      -0.797      -0.740
C(posse_de_veiculo, Treatment(reference=True))[T.False]    -0.0552      0.014     -3.914      0.000      -0.083      -0.028
C(posse_de_imovel, Treatment(reference=True))[T.False]     -0.0871      0.014     -6.312      0.000      -0.114      -0.060
idade                                                       0.0048      0.001      6.231      0.000       0.003       0.006
tempo_emprego                                               0.0610      0.001     59.126      0.000       0.059       0.063
qt_pessoas_residencia                                       0.0161      0.007      2.228      0.026       0.002       0.030
==============================================================================
Omnibus:                        1.250   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.535   Jarque-Bera (JB):                1.222
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.020   Cond. No.                         261.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [94]:
print("Modelo 1: R² ajustado =", model1.rsquared_adj, "AIC =", model1.aic, "BIC =", model1.bic)
print("Modelo 2: R² ajustado =", model2.rsquared_adj, "AIC =", model2.aic, "BIC =", model2.bic)

Modelo 1: R² ajustado = 0.18056293420046932 AIC = 258785.31252785484 BIC = 258815.02303507007
Modelo 2: R² ajustado = 0.34606436343156444 AIC = 27361.963067276585 BIC = 27413.956454903226


In [95]:
#O modelo 2 é melhor, já que seu R² é superior e seu AIC e BIC é inferior ao modelo 1.

In [96]:
# Criar matrizes de design com patsy, incluindo interações
formula = """
np.log(renda) ~ C(sexo, Treatment(0)) * idade + 
C(posse_de_veiculo, Treatment(0)) + 
C(posse_de_imovel, Treatment(1)) + 
qtd_filhos + 
C(tipo_renda, Treatment(0)) + 
C(educacao, Treatment(2)) + 
C(estado_civil, Treatment(0)) + 
C(tipo_residencia, Treatment(1)) + 
idade + 
tempo_emprego + 
qt_pessoas_residencia
"""

y, X = dmatrices(formula, data=df, return_type="dataframe")

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajustar modelos Ridge e Lasso
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_ridge)

model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"R2 Ridge: {r2_ridge}, R2 Lasso: {r2_lasso}")

R2 Ridge: 0.3673924808784039, R2 Lasso: 0.35620808655306624


In [99]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Ajustar a árvore de regressão
tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

# Fazer predições
y_pred_tree = tree.predict(X_test)

# Calcular o R2
r2_tree = r2_score(y_test, y_pred_tree)
print(f"R2 Decision Tree: {r2_tree}")

R2 Decision Tree: 0.28545285590565805
